# Analysis of students performance

## Set up block
*==> It must be executed minimum one time before executing next code blocks*

In [1]:
# === All imports ===
## global imports
import os
import sys
## spark imports
from pyspark.sql import SparkSession

# === Get data souce file ===
data_file = "./data/StudentsPerformance.csv"
if os.path.exists(data_file):
    print(f"[ok]: data source file is found: {data_file}")
else:
    print(f"[ko]: data source file is missing: {data_file}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

# === Create the Spark session
try:
    spark = SparkSession.builder.master("local").appName("# Analysis of students performance").getOrCreate()
    print("[ok]: spark session created")
except Exception as e:
    print(f"[ko]: cannot create spark session: {e}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

[ok]: data source file is found: ./data/StudentsPerformance.csv
[ok]: spark session created


## Exercise 1.1 - Exploration
### 1.1.1) Show the schema

In [2]:
df_student_performance = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .csv(data_file)
)
df_student_performance.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



### 1.1.2) Count the number of students

In [3]:
nbr_of_students = df_student_performance.count()
print("nbr of students:", nbr_of_students)

nbr of students: 1000


### 1.1.3) Display the first 10 lines

In [4]:
df_student_performance.show(10)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     